In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

/home/tl2020/Retriever/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/llama-2-7b-hf", device_map="cuda")
model = AutoModelForCausalLM.from_pretrained("meta-llama/llama-2-7b-hf", device_map="cuda")

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.87s/it]


In [3]:
from replace_llama import replace_llama
from microxcaling.mx import finalize_mx_specs

In [4]:
# generate a sentence
prompt = "The quick brown fox jumps over the lazy dog. Who said this sentence?"
input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
output = model.generate(input_ids, max_length=50, num_return_sequences=1, do_sample=False)
generated = tokenizer.decode(output[0], skip_special_tokens=True)
generated

/home/tl2020/Retriever/.venv/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/tl2020/Retriever/.venv/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'The quick brown fox jumps over the lazy dog. Who said this sentence?\nThe quick brown fox jumps over the lazy dog.\nWho said this sentence?\nThe quick brown fox jumps over the lazy dog.'

In [5]:
# Simple MX spec for MXFP6 weights+activations
mx_specs = {
    'w_elem_format': 'int8',
    'a_elem_format': 'int8',
    'block_size': 32,
    'bfloat': 16,
    'custom_cuda': True,
    # For quantization-aware finetuning, do backward pass in FP32
    'quantize_backprop': False,
}
mx_specs = finalize_mx_specs(mx_specs)
model = replace_llama(model, mx_specs).to("cuda")

replacing q_proj with microxcaling
replacing k_proj with microxcaling
replacing v_proj with microxcaling


replacing o_proj with microxcaling
replacing gate_proj with microxcaling
replacing up_proj with microxcaling
replacing down_proj with microxcaling
replacing q_proj with microxcaling
replacing k_proj with microxcaling
replacing v_proj with microxcaling
replacing o_proj with microxcaling
replacing gate_proj with microxcaling
replacing up_proj with microxcaling
replacing down_proj with microxcaling
replacing q_proj with microxcaling
replacing k_proj with microxcaling
replacing v_proj with microxcaling
replacing o_proj with microxcaling
replacing gate_proj with microxcaling
replacing up_proj with microxcaling
replacing down_proj with microxcaling
replacing q_proj with microxcaling
replacing k_proj with microxcaling
replacing v_proj with microxcaling
replacing o_proj with microxcaling
replacing gate_proj with microxcaling
replacing up_proj with microxcaling
replacing down_proj with microxcaling
replacing q_proj with microxcaling
replacing k_proj with microxcaling
replacing v_proj with micro

In [6]:
# generate a sentence
output = model.generate(input_ids, max_length=50, num_return_sequences=1, do_sample=False)
generated = tokenizer.decode(output[0], skip_special_tokens=True)
generated

'The quick brown fox jumps over the lazy dog. Who said this sentence?\nThe quick brown fox jumps over the lazy dog.\nWho said this sentence?\nThe quick brown fox jumps over the lazy dog.'

In [7]:
model.model

LlamaModel(
  (embed_tokens): Embedding(32000, 4096)
  (layers): ModuleList(
    (0-31): 32 x LlamaDecoderLayer(
      (self_attn): LlamaAttention(
        (q_proj): Linear(in_features=4096, out_features=4096, bias=True)
        (k_proj): Linear(in_features=4096, out_features=4096, bias=True)
        (v_proj): Linear(in_features=4096, out_features=4096, bias=True)
        (o_proj): Linear(in_features=4096, out_features=4096, bias=True)
        (rotary_emb): LlamaRotaryEmbedding()
      )
      (mlp): LlamaMLP(
        (gate_proj): Linear(in_features=4096, out_features=11008, bias=True)
        (up_proj): Linear(in_features=4096, out_features=11008, bias=True)
        (down_proj): Linear(in_features=11008, out_features=4096, bias=True)
        (act_fn): SiLU()
      )
      (input_layernorm): LlamaRMSNorm()
      (post_attention_layernorm): LlamaRMSNorm()
    )
  )
  (norm): LlamaRMSNorm()
)

In [10]:
type(model.model.layers[0].self_attn.k_proj)

microxcaling.mx.linear.Linear

: 